<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [17]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [32]:
# a) Consolidar y normalizar df_anio
df_list = []
for url in archivos_anio:
    df_temp = pd.read_csv(url)
    df_temp.columns = df_temp.columns.str.lower() # Normalizar a minúsculas
    df_list.append(df_temp)

df_anio = pd.concat(df_list, ignore_index=True)

# b) Explorar y limpiar df_codigos
print("Valores únicos para 'codigo_iso' en df_codigos antes de la limpieza:")
display(df_codigos['codigo_iso'].value_counts())

# Tenemos ZWE,Zimbabue ZWE,malo en el .csv
df_codigos = df_codigos.iloc[:-1]


print("\nValores únicos para 'codigo_iso' en df_codigos después de la limpieza:")
display(df_codigos['codigo_iso'].value_counts())


# c) Combinar df_anio con df_codigos
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')

print("\nDataFrame final df:")
display(df.head())
display(df.info())

Valores únicos para 'codigo_iso' en df_codigos antes de la limpieza:


,count
codigo_iso,
ZWE,2
AFG,1
ALB,1
AGO,1
ARE,1
...,...
VEN,1
WSM,1
YEM,1



Valores únicos para 'codigo_iso' en df_codigos después de la limpieza:


,count
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1



DataFrame final df:


,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


None



### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [33]:
# Estructura del DataFrame
print("Estructura del DataFrame:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print("Nombres y tipos de columnas:")
display(df.info())
#ninguna columna me parece tener un tipo inesperado

# Resumen estadístico
print("\nResumen estadístico:")
display(df.describe())
print("\nValores mínimo, máximo y promedio de 'indice':")
print(f"Mínimo: {df['indice'].min()}")
print(f"Máximo: {df['indice'].max()}")
print(f"Promedio: {df['indice'].mean()}")
#Observo que ranking tiene casi el doble de datos que índice.

print("\nValores mínimo, máximo y promedio de 'ranking':")
print(f"Mínimo: {df['ranking'].min()}")
print(f"Máximo: {df['ranking'].max()}")
print(f"Promedio: {df['ranking'].mean()}")

# Encontrar países con valores extremos en 'indice' y 'ranking'
print("\nPaíses con valores extremos en 'indice':")
display(df.loc[df['indice'].idxmin()])
display(df.loc[df['indice'].idxmax()])

print("\nPaíses con valores extremos en 'ranking':")
display(df.loc[df['ranking'].idxmin()])
display(df.loc[df['ranking'].idxmax()])


# Datos faltantes
print("\nValores nulos por columna:")
display(df.isnull().sum())
print(f"\nProporción de observaciones con valores faltantes: {df.isnull().any(axis=1).sum() / len(df) * 100:.2f}%")
print("\nColumnas con más del 30% de datos faltantes:")
missing_percentage = df.isnull().sum() / len(df) * 100
display(missing_percentage[missing_percentage > 30])


# Unicidad y duplicados
print("\nNúmero de países distintos:")
print(df['pais'].nunique())
print("\nNúmero de años distintos:")
print(df['anio'].nunique())
print("\nNúmero de filas duplicadas:")
print(df.duplicated().sum())

# Validación cruzada de columnas (inconsistencias entre pais y codigo_iso)
print("\nValidación cruzada de columnas (inconsistencias entre pais y codigo_iso):")
display(df.groupby('codigo_iso')['pais'].nunique().sort_values(ascending=False))

Estructura del DataFrame:
Número de filas: 3060
Número de columnas: 5
Nombres y tipos de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


None


Resumen estadístico:


,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000



Valores mínimo, máximo y promedio de 'indice':
Mínimo: 0.0
Máximo: 64536.0
Promedio: 205.7823160660661

Valores mínimo, máximo y promedio de 'ranking':
Mínimo: 1.0
Máximo: 121056.0
Promedio: 477.93091293620023

Países con valores extremos en 'indice':


...

,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0



Proporción de observaciones con valores faltantes: 12.97%

Columnas con más del 30% de datos faltantes:


,0



Número de países distintos:
179

Número de años distintos:
17

Número de filas duplicadas:
0

Validación cruzada de columnas (inconsistencias entre pais y codigo_iso):


,pais
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1


In [36]:
# Validación cruzada de columnas (inconsistencias entre pais y codigo_iso)

display(df['anio'].value_counts().sort_index())
display(df.groupby('codigo_iso')['pais'].value_counts())

,count
anio,
2001,180
2002,180
2003,180
2004,180
2005,180
2006,180
2007,180
2008,180
2009,180


,,count
codigo_iso,pais,
AFG,Afghanistán,17
AGO,Angola,17
ALB,Albania,17
AND,Andorra,17
ARE,Emiratos Árabes Unidos,17
...,...,...
WSM,Samoa,17
YEM,Yemen,17
ZAF,Sudáfrica,17


No parece haber inconsistencias.




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.




In [7]:

america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

print("DataFrame filtrado para países latinoamericanos:")
display(df_america.head())
display(df_america.info())

DataFrame filtrado para países latinoamericanos:


,codigo_iso,anio,indice,ranking,pais
5,ARG,2001,12.0,8.0,Argentina
7,ATG,2001,NaN,NaN,Antigua y Barbuda
20,BLZ,2001,NaN,NaN,Belize
21,BOL,2001,14.5,13.0,Bolivia
22,BRA,2001,18.8,18.0,Brasil


<class 'pandas.core.frame.DataFrame'>
Index: 476 entries, 5 to 3052
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  476 non-null    object 
 1   anio        476 non-null    int64  
 2   indice      391 non-null    float64
 3   ranking     417 non-null    float64
 4   pais        476 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 22.3+ KB


None

In [8]:
# a) Encontrar países con valores extremos usando un loop
print("Países con valores extremos (loop):")
for year in df_america['anio'].unique():
    df_year = df_america[df_america['anio'] == year].dropna(subset=['indice'])
    if not df_year.empty:
        min_indice_country = df_year.loc[df_year['indice'].idxmin()]
        max_indice_country = df_year.loc[df_year['indice'].idxmax()]
        print(f"\nAño: {year}")
        print(f"  Mayor libertad de prensa (menor indice): {min_indice_country['pais']} ({min_indice_country['indice']:.2f})")
        print(f"  Menor libertad de prensa (mayor indice): {max_indice_country['pais']} ({max_indice_country['indice']:.2f})")
    else:
        print(f"\nAño: {year} - No hay datos de 'indice' disponibles")

Países con valores extremos (loop):

Año: 2001
  Mayor libertad de prensa (menor indice): Canadá (0.80)
  Menor libertad de prensa (mayor indice): Cuba (90.30)

Año: 2002
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (1.00)
  Menor libertad de prensa (mayor indice): Cuba (97.83)

Año: 2003
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (2.00)
  Menor libertad de prensa (mayor indice): Argentina (35826.00)

Año: 2004
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (2.00)
  Menor libertad de prensa (mayor indice): Cuba (87.00)

Año: 2005
  Mayor libertad de prensa (menor indice): Bolivia (4.50)
  Menor libertad de prensa (mayor indice): Cuba (95.00)

Año: 2006
  Mayor libertad de prensa (menor indice): Canadá (4.88)
  Menor libertad de prensa (mayor indice): Cuba (96.17)

Año: 2007
  Mayor libertad de prensa (menor indice): Canadá (3.33)
  Menor libertad de prensa (mayor indice): Cuba (88.33)

Año: 2008
  Mayor libertad de prensa (menor indice): 

In [9]:
# b) Encontrar países con valores extremos usando groupby (vectorizado)
print("\nPaíses con valores extremos (groupby):")
extreme_indices = df_america.dropna(subset=['indice']).groupby('anio')['indice'].agg(['idxmin', 'idxmax'])

for year, row in extreme_indices.iterrows():
    min_country = df_america.loc[row['idxmin']]
    max_country = df_america.loc[row['idxmax']]
    print(f"\nAño: {year}")
    print(f"  Mayor libertad de prensa (menor indice): {min_country['pais']} ({min_country['indice']:.2f})")
    print(f"  Menor libertad de prensa (mayor indice): {max_country['pais']} ({max_country['indice']:.2f})")


Países con valores extremos (groupby):

Año: 2001
  Mayor libertad de prensa (menor indice): Canadá (0.80)
  Menor libertad de prensa (mayor indice): Cuba (90.30)

Año: 2002
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (1.00)
  Menor libertad de prensa (mayor indice): Cuba (97.83)

Año: 2003
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (2.00)
  Menor libertad de prensa (mayor indice): Argentina (35826.00)

Año: 2004
  Mayor libertad de prensa (menor indice): Trinidad y Tobago (2.00)
  Menor libertad de prensa (mayor indice): Cuba (87.00)

Año: 2005
  Mayor libertad de prensa (menor indice): Bolivia (4.50)
  Menor libertad de prensa (mayor indice): Cuba (95.00)

Año: 2006
  Mayor libertad de prensa (menor indice): Canadá (4.88)
  Menor libertad de prensa (mayor indice): Cuba (96.17)

Año: 2007
  Mayor libertad de prensa (menor indice): Canadá (3.33)
  Menor libertad de prensa (mayor indice): Cuba (88.33)

Año: 2008
  Mayor libertad de prensa (menor indic

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [37]:
# Construir la tabla dinámica
pivot_table_indice = pd.pivot_table(df,
                                    values='indice',
                                    index='pais',
                                    columns='anio',
                                    aggfunc='max',
                                    fill_value=0)

print("Tabla dinámica del índice máximo por país y año:")
display(pivot_table_indice.head())


Tabla dinámica del índice máximo por país y año:


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63


In [11]:
# a) ¿Qué país tiene el mayor valor de indice en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
print("\na) País con el mayor índice:")
max_indice_overall = pivot_table_indice.max().max()
country_max_indice = pivot_table_indice[pivot_table_indice == max_indice_overall].stack().idxmax()[0]
print(f"  País: {country_max_indice}")
print(f"  Índice máximo: {max_indice_overall}")

print("\nPaís con el menor índice (distinto de cero):")
min_indice_overall = pivot_table_indice[pivot_table_indice > 0].min().min()
country_min_indice = pivot_table_indice[pivot_table_indice == min_indice_overall].stack().idxmin()[0]
print(f"  País: {country_min_indice}")
print(f"  Índice mínimo (distinto de cero): {min_indice_overall}")


a) País con el mayor índice:
  País: Kosovo
  Índice máximo: 64536.0

País con el menor índice (distinto de cero):
  País: Austria
  Índice mínimo (distinto de cero): 0.5


In [12]:
# b) ¿Qué años presentan en promedio los valores de indice más altos? ¿Y los más bajos?
print("\nb) Años con promedio de índice más alto y más bajo:")
mean_indice_by_year = pivot_table_indice.mean(axis=0)
print(f"  Año con promedio de índice más alto: {mean_indice_by_year.idxmax()} ({mean_indice_by_year.max():.2f})")
print(f"  Año con promedio de índice más bajo: {mean_indice_by_year.idxmin()} ({mean_indice_by_year.min():.2f})")


b) Años con promedio de índice más alto y más bajo:
  Año con promedio de índice más alto: 2013 (449.16)
  Año con promedio de índice más bajo: 2001 (20.25)


In [13]:
# c) ¿Qué país muestra mayor variabilidad (diferencia entre su máximo y mínimo indice a lo largo del tiempo)?
print("\nc) País con mayor variabilidad en el índice:")
variability_by_country = pivot_table_indice.max(axis=1) - pivot_table_indice.min(axis=1)
country_max_variability = variability_by_country.idxmax()
print(f"  País: {country_max_variability}")
print(f"  Variabilidad máxima: {variability_by_country.max():.2f}")


c) País con mayor variabilidad en el índice:
  País: Kosovo
  Variabilidad máxima: 64536.00


In [14]:
# d) ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?
print("\nd) Países con índice constante (variabilidad cero):")
constant_indice_countries = variability_by_country[variability_by_country == 0]
if not constant_indice_countries.empty:
    display(constant_indice_countries)
else:
    print("  No hay países con índice constante a lo largo de todos los años registrados.")


d) Países con índice constante (variabilidad cero):
  No hay países con índice constante a lo largo de todos los años registrados.


In [15]:
# e) ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?
print("\ne) Países sin datos (todos los valores en la tabla pivot son 0):")
countries_with_all_zeros = pivot_table_indice[(pivot_table_indice == 0).all(axis=1)]
if not countries_with_all_zeros.empty:
    display(countries_with_all_zeros.index.tolist())
    print("\nExplicación: Estos países probablemente no tenían datos de 'indice' en los archivos originales para ninguno de los años considerados, o sus datos fueron eliminados durante el proceso de limpieza o unión.")
else:
    print("  Todos los países tienen al menos un dato distinto de cero en la tabla pivot.")


e) Países sin datos (todos los valores en la tabla pivot son 0):
  Todos los países tienen al menos un dato distinto de cero en la tabla pivot.
